In [1]:
from sas7bdat import SAS7BDAT
import pandas as pd
import pickle
import time
import datetime
import numpy as np
import statistics


In [3]:
ibes = pd.read_pickle(r"C:\Users\user\OneDrive\桌面\金融市場期末報告\ibes2.pkl")

In [6]:
ibes2 = ibes.drop(columns=['ACTDATS',"ACTTIMS",'REVTIMS','USFIRM',
  'CURRFL',
  'CURR',
  'REPORT_CURR',
  'PDF',
  'ACTUAL',
  'CURR_ACT',
  'ANNDATS_ACT',
  'ANNTIMS_ACT',
  'ACTDATS_ACT',
  'ACTTIMS_ACT','ANNTIMS','ESTIMATOR',
  'ANALYS','REVDATS'])

In [7]:
# 由ibes資料庫得到的資料型態fpi是指預估幾年後的獲利 cusip為公司代碼 value即為預測值
ibes2.head()

,OFTIC,TICKER,MEASURE,ANNDATS,FPI,CUSIP,CNAME,FPEDATS,VALUE
0,A,A,EPS,9510.0,0,02742010,AMERN MEDIC BLDG,NaN,15.00
1,A,A,EPS,9510.0,1,02742010,AMERN MEDIC BLDG,9496.0,0.01
2,A,A,EPS,9645.0,1,02742010,AMERN MEDIC BLDG,9861.0,0.60
3,A,A,EPS,9701.0,1,02742010,AMERN MEDIC BLDG,9861.0,0.30
4,A,A,EPS,9510.0,2,02742010,AMERN MEDIC BLDG,9861.0,1.75


In [9]:
# 將資料時間還原成一般格式 其中annday為宣布日期 fpeday為預估獲利的實現時間點
ibes2.insert(9,"annday",None)
ibes2.insert(10,"fpeday",None)

ibes2["fpeday"] = pd.to_timedelta(ibes2["FPEDATS"], unit='d') + pd.datetime(1960, 1, 1)
ibes2["annday"] = pd.to_timedelta(ibes2["ANNDATS"], unit='d') + pd.datetime(1960, 1, 1)
ibes2["ayear"] = ibes2["annday"].dt.year
ibes2["fyear"] = ibes2["fpeday"].dt.year

ibes2.insert(13,"base_year",None)
ibes2.insert(5,"fpi",None)

ibes2["fpi"] = ibes2[["FPI"]].astype(float)
ibes2["base_year"] = ibes2["fyear"] - ibes2["fpi"]

<ipython-input-9-c1cb07294aa0>:4: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  ibes2["fpeday"] = pd.to_timedelta(ibes2["FPEDATS"], unit='d') + pd.datetime(1960, 1, 1)
<ipython-input-9-c1cb07294aa0>:5: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  ibes2["annday"] = pd.to_timedelta(ibes2["ANNDATS"], unit='d') + pd.datetime(1960, 1, 1)


In [12]:
ibes2.head()

,OFTIC,TICKER,MEASURE,ANNDATS,FPI,fpi,CUSIP,CNAME,FPEDATS,VALUE,annday,fpeday,ayear,fyear,base_year
0,A,A,EPS,9510.0,0,0.0,02742010,AMERN MEDIC BLDG,NaN,15.00,1986-01-14,NaT,1986,NaN,NaN
1,A,A,EPS,9510.0,1,1.0,02742010,AMERN MEDIC BLDG,9496.0,0.01,1986-01-14,1985-12-31,1986,1985.0,1984.0
2,A,A,EPS,9645.0,1,1.0,02742010,AMERN MEDIC BLDG,9861.0,0.60,1986-05-29,1986-12-31,1986,1986.0,1985.0
3,A,A,EPS,9701.0,1,1.0,02742010,AMERN MEDIC BLDG,9861.0,0.30,1986-07-24,1986-12-31,1986,1986.0,1985.0
4,A,A,EPS,9510.0,2,2.0,02742010,AMERN MEDIC BLDG,9861.0,1.75,1986-01-14,1986-12-31,1986,1986.0,1984.0


In [14]:
for i in range(0,len(ibes2)):
    y1 = ibes2["fyear"][i]
    if np.isnan(y1) ==True :
        if ibes2["FPI"][i]=="0":
            ibes2["base_year"][i] = ibes2["ayear"][i]
        else:
            pass
    else:
        pass

<ipython-input-14-a6f25458bd13>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ibes2["base_year"][i] = ibes2["ayear"][i]


In [15]:
cul = ibes2["CUSIP"].tolist()
cu_li = np.unique(cul).tolist()

18375

In [17]:
# 此迴圈為將每一年分析師預估的一年後獲利&兩年後獲利的中位數

b = [[0,0,0,0,0]]
mdata = pd.DataFrame(b,columns=[0,1,2,3,4])

for i in range(1,len(cu_li)):

    cusip = cu_li[i]
    t1 = ibes2[ibes2["CUSIP"]==cusip]
    year_list = t1["base_year"].tolist()
    year_list = list(filter(None,year_list))
    year_list = np.unique(year_list).tolist()

    for j in range(0,len(year_list)):
        data = []
        year = year_list[j]
        data.append(cusip)
        data.append(year)
        t2 = t1[t1["base_year"]==year]
        te1 = t2[t2["FPI"]=="1"]
        if te1.empty:
            data.append(None)
        else:
            v1_list = te1["VALUE"].tolist()
            eps1 = statistics.median(v1_list)
            data.append(eps1)
            
        te2 = t2[t2["FPI"]=="2"]
        if te2.empty:
            data.append(None)
        else:
            v2_list = te2["VALUE"].tolist()
            eps2 = statistics.median(v2_list)
            data.append(eps2)
        
        te0 = t2[t2["FPI"]=="0"]
        if te0.empty:
            data.append(None)
        else:
            v0_list = te0["VALUE"].tolist()
            g = statistics.median(v0_list)
            data.append(g)

        da = [data]
        yda = pd.DataFrame(da)
        mdata = pd.concat([mdata,yda],axis=0)

mdata2 = mdata.iloc[1:,:]
mdata2.rename(columns={0: "cusip", 1: "year",2:"eps1",3:"eps2",4:"grow_rate"},inplace=True)

C:\Users\user\Anaconda3\anaconda3-1\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [18]:
# 最後的資料型態

mdata2.head()

,cusip,year,eps1,eps2,grow_rate
0,00000117,1982.0,16.0,None,9.0
0,00000117,1983.0,17.3,18.95,9.5
0,00000117,1984.0,17.095,18.4,9.5
0,00000117,1985.0,16.07,17.975,8.0
0,00000117,1986.0,17.5,20.0,8.0


In [ ]:
# 接著計算隱含資金成本
# 方法為第一年&第二年成長率參考ibes的預測 第三年起為ibes的grow_rate 
# 但是假設15年後的成長率會均值回歸到美國gdp成長率
# 故須先將上述資料串上股價及gdp資料

ibes_df = pd.read_pickle(r"C:\Users\user\OneDrive\桌面\金融市場期末報告\ibes_1222.pkl")
gdp = pd.read_excel(r"C:\Users\user\OneDrive\桌面\金融市場期末報告\美國GDP.xlsx")
link = pd.read_pickle(r"C:\Users\user\OneDrive\桌面\美國資料\CRSP\ccmxpf_linktable.pkl")
msf = pd.read_pickle(r"C:\Users\user\OneDrive\桌面\美國資料\CRSP\msf.pkl")
names = pd.read_pickle(r"C:\Users\user\OneDrive\桌面\美國資料\Compustat\names.pkl")

ibes_df.insert(ibes_df.shape[1],"gdp",None)
ibes_df.insert(ibes_df.shape[1],"price",None)

# 串gdp
for i in range(0,len(ibes_df)):
    print(i)
    y = ibes_df["year"][i]-1
    ibes_df["gdp"][i] = gdp[gdp["year"]==y]["gdp"].values[0]

# 串股價
names_list = np.unique(msf["CUSIP"].tolist()).tolist()
for i in range(0,len(ibes_df)):
    print(i)
    cusip = ibes_df["cusip"][i]
    year = ibes_df["year"][i]
    cs = [s for s in names_list if cusip in s]
    if cs:
        
        t1 = msf[(msf["CUSIP"]==cs[0]) & (msf["year"]==year) & (msf["month"]==6)]
        if t1.empty:
            pass
        else:
            prc = t1["PRC"].values[0]
            ibes_df["price"][i]=prc
    else:
        pass

In [ ]:
def npv(irr, cfs, yrs,b,fv,g):
    plb = []
    plb.append(0)
    plb.append(b)
    bi = g/irr
    bs = (b-bi)/14
    b2 = b - bs
    b3 = b2 - bs
    b4 = b3 - bs
    b5 = b4 - bs
    b6 = b5 - bs
    b7 = b6 - bs
    b8 = b7 - bs
    b9 = b8 - bs
    b10 = b9 - bs
    b11 = b10 - bs
    b12 = b11 - bs
    b13 = b12 - bs
    b14 = b13 - bs   
    plb.append(b2)
    plb.append(b3)
    plb.append(b4)
    plb.append(b5)
    plb.append(b6)
    plb.append(b7)
    plb.append(b8)
    plb.append(b9)
    plb.append(b10)
    plb.append(b11)
    plb.append(b12)
    plb.append(b13)
    plb.append(b14)
    
    cf = []
    for i in range(0,14):
        c = cfs[i]*(1-plb[i])/ (1. + irr) ** yrs[i]
        cf.append(c)
    fvd = fv/irr
    return np.sum(cf) + fvd/(1+irr)**14

def irr(cfs, yrs, x0):
    return np.asscalar(fsolve(npv, x0=x0, args=(cfs, yrs,b,fv,g)))

In [6]:
# 最後的icc即為隱含資金成本

for i in range(0,len(ibes_df)):
    print(i)
    eps = []
    pp = ibes_df["price"][i]
    if pp == None:
        pass
    else:
        
        p = -pp
        eps.append(p)
        eps.append(ibes_df["eps1"][i])
        eps.append(ibes_df["eps2"][i])
        for j in range(5,18):
            eps.append(ibes_df.iloc[i,j])
            
        cash_flow = np.array(eps)
        years = np.arange(15)
        b = ibes_df["pay_out_ratio"][i]
        fv = ibes_df["y15"][i]
        g = ibes_df["gdp"][i]
        a = irr(cfs=cash_flow, yrs=years, x0=0.10)
        ibes_df["icc"][i] = a

ibes_df

,cusip,year,eps1,eps2,grow_rate,y3,y4,y5,y6,y7,...,y10,y11,y12,y13,y14,y15,pay_out_ratio,gdp,price,icc
44,00016510,1985.0,0.8,0.85,15.0,0.9775,1.121013,1.282104,1.462434,1.663763,...,2.412792,2.717642,3.053723,3.423352,3.828928,4.272927,0.186057,0.11109,6.5,0.184406
45,00020910,1987.0,0.5,0.715,20.0,0.858,1.014575,1.183512,1.363324,1.55232,...,2.155902,2.363061,2.570243,2.77586,2.978482,3.176859,0.111996,0.055451,12.0,0.116963
46,00020910,1988.0,0.74,0.97,18.5,1.14945,1.345712,1.557778,1.784347,2.02387,...,2.802197,3.075134,3.351575,3.62965,3.907591,4.183756,0.264743,0.060202,16.0,0.110202
47,00020910,1989.0,0.93,1.17,30.0,1.521,1.935627,2.415093,2.95869,3.563821,...,5.697867,6.492363,7.314964,8.157146,9.01052,9.867081,-0.018245,0.078492,24.25,0.138644
48,00020910,1990.0,1.07,1.23,20.0,1.476,1.751842,2.057764,2.393545,2.758504,...,4.015365,4.482134,4.968994,5.473344,5.992454,6.523528,-0.014532,0.077381,33.0,0.089919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57748,Y6476W10,2003.0,2.49,2.565,15.0,2.94975,3.3473,3.752636,4.16093,4.567694,...,5.741177,6.106951,6.456535,6.788613,7.10233,7.397228,0.284159,0.03352,6.16,0.399058
57749,Y6476W10,2004.0,2.88,2.85,12.0,3.192,3.550619,3.924095,4.31054,4.707977,...,5.945987,6.367214,6.789523,7.211132,7.630371,8.045702,-0.07782,0.047712,11.9,0.315415
57751,Y8564W10,2001.0,0.775,1.8,10.4,1.9872,2.186947,2.399411,2.624709,2.862908,...,3.6548,3.944233,4.246127,4.560243,4.886299,5.223967,0.24454,0.064554,40.02,0.06815
57752,Y8564W10,2002.0,3.5325,3.575,13.0,4.03975,4.515017,4.995725,5.477075,5.95466,...,7.32821,7.756816,8.167383,8.558604,8.929611,9.27992,0.082254,0.032129,36.91,0.139003
